# Digikala data for Apple, Samsung, Xiaomi

In [ ]:
! pip install selenium
! pip install pandas==1.5.3
! pip install unidecode

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from unidecode import unidecode
import time

In [3]:
# this ignores the FutureWarning warning from pandas.append() 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### opening digikala

In [17]:
driver = webdriver.Chrome()

In [5]:
url = "https://www.digikala.com/search/category-mobile-phone/"
apple_url = url + "apple/"
xiaomi_url = url + "xiaomi/"
samsung_url = url + "samsung/"

getting links of 200 apple product if availible

In [25]:
driver.maximize_window()
driver.get(apple_url)

In [26]:
# choose most selled
driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[1]/div/div/div/div[2]/span[3]""").click()

In [27]:
apple_links = []
# link_count = how many data you want
link_count = 200
y = 480

for _ in range(link_count // 4):
    cards = driver.find_elements(By.XPATH, f"/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[2]/div/a")
    for card in cards:
        if "ناموجود" not in card.text and "توقف تولید" not in card.text and "به‌زودی" not in card.text:
            if (link := card.get_attribute("href")) not in apple_links:
                apple_links.append(link)

    driver.execute_script(f"window.scrollTo(0,{y})")
    y += 480
    time.sleep(2)

In [75]:
len(apple_links)

31

getting data from each link

In [86]:
apple_df = pd.DataFrame(columns=["brand", 'model', 'off_price',"price", "storage", "ram", "color", "link"])
apple_df

,brand,model,off_price,price,storage,ram,color,link


In [87]:
i = 1
for link in apple_links:
    print(i)
    i += 1
    driver.get(link)
    time.sleep(3)
    # Loading all the page
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0,0)")
    time.sleep(2)

    wait = WebDriverWait(driver, 10)
    # getting the title
    title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, """h1.text-h4"""))).text

    # color
    try:
        # selecting first color
        color = driver.find_element(By.CSS_SELECTOR, """div.styles_InfoSectionVariationColor__pX_3M:nth-child(1)""")
        color.click()
        color = color.text
    except:
        color = "NAN"

    more = wait.until(EC.visibility_of_element_located((By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/span/span""")))
    try:
        more.location_once_scrolled_into_view
        # driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)
        more.click()
    except:
        driver.execute_script(f"window.scrollTo({more.location['x']},{more.location['y']-300})")
        time.sleep(3)    
        more.click()


    # getting infos from specs
    ram = "NAN"
    storage = "NAN"
    model = "NAN"

    infos = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[1]/section/div[2]/div/div""")

    for info in infos:
        if "حافظه داخلی" in info.text:
            storage_info = infos.index(info)
            storage = unidecode(infos[storage_info].text.split("\n")[1].split(" ")[0])

        if "مقدار RAM" in info.text:
            ram_info = infos.index(info)
            ram = infos[ram_info].text.split("\n")[1]

        if "مدل" in info.text:
            model_info = infos.index(info)
            model = unidecode(infos[model_info].text.split("\n")[1])

    
    # Get price
    try:
        prices = driver.find_elements(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div/div[2]/div/div[1]/div""")
        price_infos = []
        for info in prices:
            if "افزودن به سبد" in info.text:
                price_infos = info.text.split("\n")

        if len(price_infos) > 2:
            # with off
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            try:
                off_price = unidecode(price_infos[2].replace(",", ""))
                off_price = int(off_price)
            except ValueError:
                off_price = "NAN"
        else:
            price = unidecode(price_infos[0].replace(",", ""))
            price = int(price)
            off_price = "NAN"
    except:
        # Not availible
        continue
    
    data = {
        "brand" : "apple",
        "title" : title,
        "model" : model,
        'price' : price,
        "off_price" : off_price,
        "color" : color,
        "storage" : storage,
        "ram" : ram,
        "link" : link
    }
    
    apple_df = apple_df.append(data, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [88]:
apple_df

,brand,model,off_price,price,storage,ram,color,link,title
0,apple,iPhone 13 CH,NAN,37450000,128,چهار گیگابایت,مشکی,https://www.digikala.com/product/dkp-8366616/%...,گوشی موبایل اپل مدل iPhone 13 CH دو سیم‌ کارت ...
1,apple,iPhone 13 CH,NAN,44945000,256,چهار گیگابایت,مشکی,https://www.digikala.com/product/dkp-9125271/%...,گوشی موبایل اپل مدل iPhone 13 CH دو سیم‌ کارت ...
2,apple,iPhone 11 LZA,NAN,30098000,128,چهار گیگابایت,سفید,https://www.digikala.com/product/dkp-10759493/...,گوشی موبایل اپل مدل iPhone 11 LZA تک سیم‌ کارت...
3,apple,iPhone 13 Pro,NAN,74990000,ykh,شش گیگابایت,سبز,https://www.digikala.com/product/dkp-10741540/...,گوشی موبایل اپل مدل iPhone 13 Pro تک سیم کارت ...
4,apple,iPhone 13 Pro Max,NAN,88400000,ykh,شش گیگابایت,سبز,https://www.digikala.com/product/dkp-10741745/...,گوشی موبایل اپل مدل iPhone 13 Pro Max تک سیم‌ ...
5,apple,iPhone 13 Pro Max LLA,79900000,81900000,128,شش گیگابایت,سفید,https://www.digikala.com/product/dkp-10632768/...,گوشی موبایل اپل مدل iPhone 13 Pro Max LLA تک س...
6,apple,iPhone 13 Pro CH,71442000,72900000,256,شش گیگابایت,سبز,https://www.digikala.com/product/dkp-10443781/...,گوشی موبایل اپل مدل iPhone 13 Pro CH دو سیم‌ ک...
7,apple,iPhone 13 CH,NAN,39000000,128,چهار گیگابایت,مشکی,https://www.digikala.com/product/dkp-12085945/...,گوشی موبایل اپل مدل iPhone 13 CH دو سیم‌ کارت ...
8,apple,iPhone SE 2022 JA,NAN,27000000,128,چهار گیگابایت,مشکی,https://www.digikala.com/product/dkp-9759409/%...,گوشی موبایل اپل مدل iPhone SE 2022 JA تک سیم ک...
9,apple,iPhone 11 ZPA,NAN,29900000,128,چهار گیگابایت,مشکی,https://www.digikala.com/product/dkp-9950935/%...,گوشی موبایل اپل مدل iPhone 11 ZPA تک سیم‌ کارت...


In [89]:
apple_df.to_csv("apple_data.csv")